In [1]:
%pylab inline
import pandas as pd
import numpy as np
import cv2
import pylab
import glob
from pydicom import dicomio
#import png
from PIL import Image, ImageEnhance, ImageFilter
import re

Populating the interactive namespace from numpy and matplotlib


In [3]:
for name3 in glob.glob('train/[0-9]*'):
    print name3
    patient=re.findall('\/[0-9]*',name3) #Extracts patient number from file name
    print patient
    for name2 in glob.glob(name3+'/study/sax_[0-9]*'):
        #sax = re.findall('sax_[0-9]*',name2) #Extracts the sax number, ex. sax_10
        k=0
        new_im = Image.new('RGB', (3000,100)) #initializes a new image
        minmax_im = Image.new('RGB', (400,200))
        brightness=np.zeros(40)
        files=[] #keeps a list of which files we have been through
        l=0
        for name in glob.glob(name2+'/IM-[0-9]*-[0-9]*.dcm'): #this is the full file name
            files.append(name)
            sax = re.findall('sax_[0-9]*',name)
            ds= dicomio.read_file(name) #reads the .dcm file
            
            array=ds.pixel_array #extracts the image as a pixel_array
            array=array/(np.max(array)/255.0) #normalizing the pixels to get a better image
            array=array.astype('uint8')
            im = Image.fromarray(array) #creates the actual image from the image array
            enhancer=ImageEnhance.Brightness(im) #creates the brightness enhancer
            im=enhancer.enhance(1.5) #enhances brightness
            enhancer=ImageEnhance.Sharpness(im) #creates the sharpness enhancer
            im=enhancer.enhance(2) #enhances sharpness
            
            #array=ds.pixel_array
            #for i in range(shape(array)[0]):
            #    for j in range(shape(array)[1]):
            #        if array[i,j]>255:
            #            array[i,j]=255
            brightness[l]=(array>50).sum() #counts the number of pixels above 50 (higher number=brighter)
            l=l+1
            #array=array.astype('uint8')
            #im = Image.fromarray(array)
            im.thumbnail((100,100))
            new_im.paste(im, (k,l)) #pastes this thumbnail into part of new_im (new_im will contain all the images for a certain patient and sax)
            k=k+100
        new_im.save('Breath cycles/'+patient[0]+sax[0]+'.png')
        brightness=brightness.astype(int)
        brightness=brightness[brightness != 0]

        ######Find Image with Minimum Brightness######################
        #(logic behind this: in each image, the heart should be the only thing changing/moving. 
        #Thus, the image in a given sax with the largest number of bright pixels should be the image where the heart is the most expanded.
        #And, the image with the smallest number of bright pixels should be the image where the heart is the most contracted.)
        
        minind=np.argmin(brightness) #finds index of image with min brightness
        ds= dicomio.read_file(files[minind])
        array=ds.pixel_array
        array=array/(np.max(array)/255.0)
        array=array.astype('uint8')
        im = Image.fromarray(array)

        enhancer=ImageEnhance.Brightness(im)
        im=enhancer.enhance(1.5)
        enhancer=ImageEnhance.Sharpness(im)
        im=enhancer.enhance(2)
        #for i in range(shape(array)[0]):
        #        for j in range(shape(array)[1]):
        #            if array[i,j]>255:
        #                array[i,j]=255
        #array=array.astype('uint8')
        #im = Image.fromarray(array)
        im.save('Mins/'+patient[0]+sax[0]+'_min.png') #save this image as the minimum for that patient and sax number   

        #######Find Image with Maximum Brightness#########################
        
        maxind=np.argmax(brightness)
        ds= dicomio.read_file(files[maxind])
        array=ds.pixel_array
        array=array/(np.max(array)/255.0)
        array=array.astype('uint8')
        im2 = Image.fromarray(array)
        enhancer=ImageEnhance.Brightness(im2)
        im2=enhancer.enhance(1.5)
        enhancer=ImageEnhance.Sharpness(im2)
        im2=enhancer.enhance(2)
        #array=ds.pixel_array
        #for i in range(shape(array)[0]):
        #        for j in range(shape(array)[1]):
        #            if array[i,j]>255:
        #                array[i,j]=255
        #array=array.astype('uint8')
        #im = Image.fromarray(array)
        im2.save('Maxs/'+patient[0]+sax[0]+'_max.png')
        
        #####Saves the min and max images side by side for comparison purposes############
        im.thumbnail((200,200))
        im2.thumbnail((200,200))
        minmax_im.paste(im, (0,1))
        minmax_im.paste(im2, (200,1))
        minmax_im.save('MinMax/'+patient[0]+sax[0]+'_max.png')

train/1
['/1']
train/10
['/10']
train/100
['/100']
train/101
['/101']
train/102
['/102']
train/103
['/103']
train/104
['/104']
train/105
['/105']
train/106
['/106']
train/107
['/107']
train/108
['/108']
train/109
['/109']
train/11
['/11']
train/110
['/110']
train/111
['/111']
train/112
['/112']
train/113
['/113']
train/114
['/114']
train/115
['/115']
train/116
['/116']
train/117
['/117']
train/118
['/118']
train/119
['/119']
train/12
['/12']
train/120
['/120']
train/121
['/121']
train/122
['/122']
train/123
['/123']


IndexError: index 40 is out of bounds for axis 0 with size 40